In [8]:
import datetime
import firecloud.api as FAPI
import json
import os
import pandas as pd
import pprint
import re
import requests


# Anvil Tools
#https://github.com/broadinstitute/horsefish/blob/master/scripts/anvil_tools/utils.py
    
# Schema
# https://github.com/broadinstitute/firecloud-orchestration/blob/develop/src/main/resources/library/attribute-definitions.json
# https://github.com/broadinstitute/firecloud-orchestration/blob/219bdb1a7c4c85df5350f27be9292bb66233a355/src/main/resources/library/attribute-definitions.json

#  "consentCodes": {
#     "GRU": "For health/medical/biomedical purposes and other biological research, including the study of population origins or ancestry.",
#     "HMB": "Use of the data is limited to health/medical/biomedical purposes, does not include the study of population origins or ancestry.",
#     "DS": "Use of the data must be related to a particular disease.",
#     "NCU": "Use of the data is limited to non-commercial use.",
#     "NPU": "Use of the data is limited to not-for-profit organizations.",
#     "NMDS": "Use of the data includes methods development research (e.g., development of software or algorithms) ONLY within the bounds of other specified data use limitations.",
#     "NAGR": "Use of the data includes aggregate level analysis to address research questions (e.g., determine variant frequencies across the general population) ONLY within the bounds of other specified data use limitations.",
#     "NCTRL": "Data can be used as a control set ONLY within the bounds of other specified data use limitations (e.g. only for cancer studies).",
#     "RS-G": "Use of the data is limited to studies of particular gender.",
#     "RS-PD": "Use of the data is limited to pediatric research."

#   "required": ["library:datasetName", "library:datasetVersion", "library:datasetDescription", "library:datasetCustodian",
#     "library:datasetDepositor", "library:contactEmail", "library:datasetOwner", "library:institute", "library:indication",
#     "library:numSubjects", "library:projectName", "library:dataCategory", "library:datatype", "library:dataUseRestriction",
#     "library:studyDesign", "library:requiresExternalApproval", "library:useLimitationOption"],
#   "oneOf": [{
#     "required": ["library:GRU", "library:HMB", "library:NCU", "library:NPU", "library:NMDS", "library:NAGR",
#       "library:NCTRL", "library:RS-G", "library:RS-PD", "library:useLimitationOption", "library:IRB"],
#     "anyOf": [
#       {
#         "properties": {"library:GRU": {"enum": [true]}}
#       },
#       {
#         "properties": {"library:HMB": {"enum": [true]}}
#       },
#       {
#         "required": ["library:DS"],
#         "properties": {"library:DS": {"minItems": 1}}
#       }
#     ],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["questionnaire"]
#       }
#     }
#   }, {
#     "required": ["library:orsp", "library:useLimitationOption"],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["orsp"]
#       }
#     }
#   }, {
#     "required": ["library:useLimitationOption"],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["skip"]
#       }
#     }
#   }],


print(FAPI.whoami())
pp = pprint.PrettyPrinter(indent=4)

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"
# CATALOG_WORKSPACE_NAMESPACE = "clever-canary-anvil-dev"

API_ROOT = "https://api.firecloud.org/api"


#TEST_WORKSPACE = 'AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome_copy'

TEST_WORKSPACE = 'AnVIL_CMG_Broad_Blood_Gazda_WES_copy'


def getDefaultAccessToken():
    result = !gcloud auth application-default print-access-token
    return result[0]


ACCESS_TOKEN = getDefaultAccessToken()

print("Copy files:")
# print(WORKSPACE_BUCKET)
!gsutil cp $WORKSPACE_BUCKET/notebooks/AnVILCatalogWorkspacesPublicMetadata-2022-03-11.csv  .
print("List all files:")
!ls 

## Read in metadata list
metadata = pd.read_csv('AnVILCatalogWorkspacesInput-2022-03-11.csv',keep_default_na=False)
metadata = metadata[["name",
                     "consortium",
                     "phsId",
                     "library:indication",
                     "library:datatype",
                     "library:studyDesign",
                     "library:dataUseRestriction"]]
metadata.set_index("name", inplace=True)

## Filter out workspaces without write access
workspaces = FAPI.list_workspaces().json()

reader = set()
writer = set()
for w in workspaces:
    name = w['workspace']['name']
    if w['accessLevel'] == "READER":
        reader.add(name)
    if w['accessLevel'] == "WRITER" or w['accessLevel'] == "OWNER":
        writer.add(name)
        
def getName(workspace):
    return workspace['workspace']['name']        

workspaceByName = {}
for w in workspaces:
    workspaceByName[getName(w)]=w


pet-117272931645288568532@terra-e36fcccd.iam.gserviceaccount.com
Copy files:
CommandException: No URLs matched: gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/notebooks/AnVILCatalogWorkspacesPublicMetadata-2022-03-11.csv
List all files:
 1000G-high-coverage-2019_2022-03-10T07:30:15.619581.json
'Add Workspace Writer.ipynb'
 AnVILCatalogWorkspacesInput-2022-03-07.csv
 AnVILCatalogWorkspacesInput-2022-03-08.csv
 AnVILCatalogWorkspacesInput-2022-03-10.csv
 AnVILCatalogWorkspacesInput-2022-03-11.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-02-25.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-08.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-09.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-10.csv
 AnVILCatalogWorkspacesPublicMetadata-2022-03-11.csv
 AnVIL_ccdg_asc_ndd_daly_talkowski_ac-boston_asd_exome_2022-03-10T07:09:16.963605.json
 AnVIL_ccdg_asc_ndd_daly_talkowski_AGRE_asd_exome_2022-03-10T07:29:12.251996.json
 AnVIL_ccdg_asc_ndd_daly_talkowski_AGRE-FEMF_asd_exome_2022-03-10T07

In [9]:
def getUpdateAttributesURL(namespace,workspace):
    return  API_ROOT+"/library/"+ namespace + "/" + workspace + "/metadata"

def getAPIHeaders():
    headers = {
        'accept': 'application/json',
        'Authorization' : 'Bearer '+ACCESS_TOKEN
    }
    return headers


# Replaces the set the tags on a workspace to the given list of tags
def updateWorkspaceAttributes(namespace,workspace,attributes):        
    url = getUpdateAttributesURL(namespace,workspace)
    headers = getAPIHeaders()
    resp = requests.put(url, json=attributes, headers=headers)
    print(resp.status_code)
    print(resp.text)

# 
def getWorkspaceAttributes(namespace,workspace):        
    url = getUpdateAttributesURL(namespace,workspace)
    headers = getAPIHeaders()
    resp = requests.get(url, headers=headers)
    print(resp.status_code)
    pp.pprint(json.dumps(resp.json()))
    return resp.json()
    
def backupWorkspace(w):
   
    # Create the filename
    name = w['workspace']['name']
    timestampStr = datetime.datetime.now().isoformat()
    fileName = name+'_'+timestampStr+'.json'
    
    # Save the to the local cloud environment
    with open(fileName, 'w') as fp:
        json.dump(w, fp,  indent=4)
        
    # Copy the file to the workspace bucket
    !gsutil cp ./$fileName $WORKSPACE_BUCKET/backups/

    
# publish a workspace to the Data Library in FireCloud
def publish_workspace_to_data_library(workspace_name, project="anvil-datastorage"):
    """Publish workspace to Firecloud Data Library."""

    # Library/publishLibraryWorkspace
    uri = f"https://api.firecloud.org/api/library/{project}/{workspace_name}/published"

    # Get access token and and add to headers for requests.
    # -H  "accept: application/json" -H  "Authorization: Bearer [token]"
    headers = {"Authorization": "Bearer " + ACCESS_TOKEN, "accept": "application/json"}

    # capture API response and status_code
    response = requests.post(uri, headers=headers)
    status_code = response.status_code

    # publishing fail
    if status_code not in [200, 204]:
        print(f"WARNING: Failed to publish workspace to Data Library: {project}/{workspace_name}.")
        print("Please see full response for error:")
        print(response.text)
        return False, response.text

    # publishiing success
    print(f"Successfully published {project}/{workspace_name} to Data Library.")
    return True, response.text

"""
For the catalog we want to set:

1.  consent code - how?
2. Library:indication -  "library:indication": "Poor health",
3. library:studyDesign - "library:studyDesign": "Case/Control"
4. library:dataType -  "library:datatype": ["Whole Exome", "Whole Genome"],

consortia tag
phsid tag

maybe make numSubjects match the workspace data


TODO: 


1. Combine different spreadsheets dave using + tags

1. Strip down input to only the columns we care about

1. Validate the spreadsheet contains only allowed values for study design and data type - nope just show wht the  have for now
and try to get to standardise maybe on values from TIM

DataType
Whole Exome, Whole Genome, Exome Panel, Genotyping Array, RNA-Seq, Methylation Array
     
Study Design
Case/Control, Tumor/Normal, Trio/Twin/Family, Prospective Longitudinal Cohort, Cases only - somatic, Cases only - germline, Controls
     
"""

# testAttributes = {
    
#     "library:datasetName": "Workspace Name",
#     "library:datasetVersion": "1.1",
#     "library:datasetDescription": "Cohort Description",
#     "library:datasetCustodian": "Sam Sneed",
#     "library:datasetDepositor": "Larry Sneed",
#     "library:contactEmail": "sam@sneed.com",
#     "library:datasetOwner": "Virgina Sneed",
#     "library:institute": ["Scool of Hard Knocks"],
#     "library:indication": "Poor health",
#     "library:numSubjects": 123,
#     "library:projectName": "TopMed",
#     "library:dataCategory": ["Simple Nucleotide Variation (SNVs), CNV"],
#     "library:datatype": ["Whole Exome", "Whole Genome"],
#     "library:dataUseRestriction": "HMB-ABC-123",
#     "library:studyDesign": "Case/Control",
#     "library:requiresExternalApproval": False,
#     "library:useLimitationOption": "skip",
#     "library:GRU": True,

# }   


# We will set:
#     "library:indication": "Poor health",
#     "library:datatype": ["Whole Exome", "Whole Genome"],
#     "library:dataUseRestriction": "HMB-ABC-123",
#     "library:studyDesign": "Case/Control",

#if they are different


"""
   DataUseRestriction:  -> DS-BFD-MDS
    Indication: mendelian disorders -> Mendelian
    DataTypes: Whole Genome -> Exome
    StudyDesign:  -> Parent-Offspring Trios
"""

## Update workspace attributes
print(CATALOG_WORKSPACE_NAMESPACE)   
length = 0        
for name in metadata.index:
    if name in workspaceByName:
        length = length+1
        if length <= 300:
            continue 
#         if length > 300:
#             break        
        print("##########################")
        print(name) 
        backupWorkspace(w)
    
        update_attrs = [

            {
                "name": "library:dataUseRestriction",
                "value":  metadata.at[name, "library:dataUseRestriction"]
            },
             {
                "name": "library:indication",
                "value": metadata.at[name, "library:indication"]
            },
            {
                "name": "library:datatype",
                "value": metadata.at[name, "library:datatype"].split(",")
            },
             {
                "name": "library:studyDesign",
                "value": metadata.at[name, "library:studyDesign"]
            },

        ]


 
        current_attrs = getWorkspaceAttributes(CATALOG_WORKSPACE_NAMESPACE,name)
        next_attrs = current_attrs.copy()

        for update in update_attrs:
            key = update["name"]
            ## use set for unordered comparison of lists
            if set(next_attrs[key]) != set(update['value']):
                next_attrs[key] = update['value']
        pp.pprint(next_attrs)    

        #updateWorkspaceAttributes(CATALOG_WORKSPACE_NAMESPACE,name,next_attrs)

        publish_workspace_to_data_library(name)

200
# {"attributes":{"library:requiresExternalApproval":false,"library:discoverableByGroups":{"itemsType":"AttributeValue","items":["GP_DataDelivery_TaskTeam"]},"library:dulvn":4,"library:GR":true,"library:studyDesign":"Case/Control","description":"ANVIL_CMG_BROAD_BRAIN_ENGLE_WES","library:published":true,"library:indication":"Poor health","library:contactEmail":"sam@sneed.com","library:numSubjects":123,"library:datasetOwner":"Virgina Sneed","library:datatype":{"itemsType":"AttributeValue","items":["Whole Exome","Whole Genome"]},"library:datasetCustodian":"Sam Sneed","library:projectName":"TopMed","library:institute":{"itemsType":"AttributeValue","items":["Scool of Hard Knocks"]},"tag:tags":{"itemsType":"AttributeValue","items":["Access WS","DUL:GSR:notAllowed","GRCh38/hg38","Consortium: CMG","dbGaP: phs001272"]},"library:dataUseRestriction":"HMB-ABC-123","library:datasetDepositor":"Larry Sneed","library:datasetVersion":"1.1","library:datasetName":"Workspace Name","library:dataCategory":{"itemsType":"AttributeValue","items":["Simple Nucleotide Variation (SNVs), CNV"]},"library:useLimitationOption":"skip","library:datasetDescription":"Cohort Description"},"authorizationDomain":[{"membersGroupName":"AUTH_AnVIL_CMG_Broad_Brain_Engle_WES"}],"billingAccount":"billingAccounts/00C5C3-F6B804-1667EC","bucketName":"fc-secure-76700a19-3005-4f2e-a595-abed41d306d4","completedCloneWorkspaceFileTransfer":"2022-03-04T22:43:02.100Z","createdBy":"dave@clevercanary.com","createdDate":"2022-03-04T22:42:55.280Z","googleProject":"terra-51086c51","googleProjectNumber":"568298150090","isLocked":false,"lastModified":"2022-03-04T22:42:56.769Z","name":"Clever_Canary_ANVIL_CMG_BROAD_BRAIN_ENGLE_WES_TEST","namespace":"anvil-datastorage","workflowCollectionName":"76700a19-3005-4f2e-a595-abed41d306d4","workspaceId":"76700a19-3005-4f2e-a595-abed41d306d4","workspaceVersion":"v2"}




anvil-datastorage
##########################
AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMR_GRU_NPU_WES
Copying file://./AnVIL_CCDG_Baylor_CVD_HemStroke_Regards_DS_WGS_2022-03-11T23:03:42.421124.json [Content-Type=application/json]...
/ [1 files][  4.1 KiB/  4.1 KiB]                                                
Operation completed over 1 objects/4.1 KiB.                                      
200
('{"library:requiresExternalApproval": false, "library:discoverableByGroups": '
 '["GP_DataDelivery_TaskTeam"], "library:dulvn": 4, "library:studyDesign": '
 '"NA", "library:NPU": true, "library:indication": "NA", '
 '"library:contactEmail": "ccusick@broadinstitute.org", "library:numSubjects": '
 '1086, "library:datasetOwner": "NA", "library:datatype": ["Exome"], '
 '"library:datasetCustodian": "ccusick@broadinstitute.org", '
 '"library:projectName": "NA", "library:GRU": true, "library:institute": '
 '["Broad"], "library:technology": ["Illumina"], "library:dataUseRestriction": '
 '"NA", "library:datase

200
{"attributes":{"library:requiresExternalApproval":false,"library:discoverableByGroups":{"itemsType":"AttributeValue","items":["GP_DataDelivery_TaskTeam"]},"library:NMDS":true,"library:dulvn":4,"library:studyDesign":"TBD","description":"","library:published":true,"library:NCTRL":true,"library:indication":"epilepsy","library:contactEmail":"ccusick@broadinstitute.org","library:numSubjects":479,"library:datasetOwner":"NA","library:datatype":{"itemsType":"AttributeValue","items":["Genotyping Array"]},"library:datasetCustodian":"ccusick@broadinstitute.org","library:projectName":"NA","library:institute":{"itemsType":"AttributeValue","items":["Broad"]},"library:technology":{"itemsType":"AttributeValue","items":["Illumina"]},"tag:tags":{"itemsType":"AttributeValue","items":["Access WS","DUL:GSR:allowed","GRCh37/hg19","Infinium Global Screening Array + Multi Disease Processing","Jul 30, 2021","PDO-13210","RP-1229","Whole Genome Genotyping","Consortium: CCDG","dbGaP: TBD"]},"library:dataUseRe

200
{"attributes":{"library:HMB":"","library:lmsvn":"6","library:requiresExternalApproval":"FALSE","library:discoverableByGroups":{"itemsType":"AttributeValue","items":[]},"library:NMDS":"","library:dulvn":"4","library:studyDesign":"Case-Control","library:NCU":"","library:cohortCountry":"","description":"AnVIL_CCDG_Broad_AI_IBD_McGovern_WGS","workspace-column-defaults":"{\"participant\":{\"shown\":[\"collaborator_participant_id\",\"gender\"],\"hidden\":[\"participant_id\",\"participant\"]},\"sample\":{\"shown\":[\"project\",\"collaborator_sample_id\",\"version\",\"data_type\",\"pdo\",\"release_date\",\"reference_sequence_name\",\"collaborator_participant_id\",\"cram_path\",\"crai_path\",\"md5_path\",\"root_sample_id\",\"mean_coverage\",\"adapter_rate\",\"pf_hq_aligned_q20_bases\",\"pf_hq_aligned_reads\",\"pf_mismatch_rate\",\"pf_noise_reads\",\"pf_reads\",\"pf_reads_aligned\",\"total_reads\",\"mean_read_length\",\"pf_reads_rate\",\"pf_reads_aligned_rate\",\"pf_aligned_bases\",\"reads_a

200
{"attributes":{"library:requiresExternalApproval":false,"library:discoverableByGroups":{"itemsType":"AttributeValue","items":["GP_DataDelivery_TaskTeam"]},"library:dulvn":4,"library:studyDesign":"Case Set","description":"AnVIL_CCDG_Broad_NP_Epilepsy_USAUPN_Marsh_GRU_NPU_WES","workspace-column-defaults":"{\"participant\":{\"shown\":[\"collaborator_participant_id\",\"gender\"],\"hidden\":[\"participant_id\",\"participant\"]},\"sample\":{\"shown\":[\"project\",\"collaborator_sample_id\",\"version\",\"data_type\",\"pdo\",\"release_date\",\"reference_sequence_name\",\"collaborator_participant_id\",\"cram_path\",\"crai_path\",\"md5_path\",\"root_sample_id\",\"mean_coverage\",\"adapter_rate\",\"pf_hq_aligned_q20_bases\",\"pf_hq_aligned_reads\",\"pf_mismatch_rate\",\"pf_noise_reads\",\"pf_reads\",\"pf_reads_aligned\",\"total_reads\",\"mean_read_length\",\"pf_reads_rate\",\"pf_reads_aligned_rate\",\"pf_aligned_bases\",\"reads_aligned_in_pairs\",\"exc_dupe_rate\",\"contamination_rate\",\"gen

Successfully published anvil-datastorage/AnVIL_CCDG_Broad_NP_Epilepsy_IRLRCI_GRU_IRB_WES to Data Library.
##########################
AnVIL_CCDG_WASHU_PAGE
Copying file://./AnVIL_CCDG_Baylor_CVD_HemStroke_Regards_DS_WGS_2022-03-11T23:04:13.220935.json [Content-Type=application/json]...
/ [1 files][  4.1 KiB/  4.1 KiB]                                                
Operation completed over 1 objects/4.1 KiB.                                      
200
('{"library:HMB": "", "library:lmsvn": "6", '
 '"library:requiresExternalApproval": "", "library:NMDS": "", "library:dulvn": '
 '"", "library:studyDesign": "", "library:NCU": "", "library:cohortCountry": '
 '"", "library:NPU": "", "library:NCTRL": "", "library:indication": '
 '"Unspecified", "library:contactEmail": "help@lists.anvilproject.org", '
 '"library:numSubjects": "", "library:datasetOwner": "", "library:datatype": '
 '[""], "library:primaryDiseaseSite": "", "library:datasetCustodian": "", '
 '"library:projectName": "", "library:GRU"

...

Successfully published anvil-datastorage/AnVIL_NIMH_Broad_WGSPD1_McCarroll_Escamilla_DS_WGS to Data Library.
##########################
AnVIL_NIMH_Broad_WGSPD1_McCarroll_Braff_DS_WGS
Copying file://./AnVIL_CCDG_Baylor_CVD_HemStroke_Regards_DS_WGS_2022-03-11T23:22:45.402584.json [Content-Type=application/json]...
/ [1 files][  4.1 KiB/  4.1 KiB]                                                
Operation completed over 1 objects/4.1 KiB.                                      
200
('{"library:lmsvn": 6, "library:requiresExternalApproval": false, '
 '"library:dulvn": 4, "library:studyDesign": "Case/Control", '
 '"library:coverage": "11x-20x", "library:diseaseOntologyID": '
 '"http://purl.obolibrary.org/obo/DOID_5419", "library:indication": '
 '"schizophrenia", "library:contactEmail": "help@lists.anvilproject.org", '
 '"library:numSubjects": 864, "library:diseaseOntologyLabel": "schizophrenia", '
 '"library:datasetOwner": "Steve McCarroll", "library:datatype": ["Whole '
 'Genome"], "library:d

200
{"attributes":{"library:HMB":"FALSE","library:lmsvn":"6","library:requiresExternalApproval":"FALSE","library:NMDS":"FALSE","library:dulvn":"4","library:studyDesign":"Case-control","library:NCU":"FALSE","library:cohortCountry":"United States, Portugal","description":"","library:NPU":"FALSE","library:NCTRL":"FALSE","library:indication":"Schizophrenia and Bipolar Disorder cases and controls","library:contactEmail":"help@lists.anvilproject.org","library:numSubjects":"8084","library:datasetOwner":"Steve McCarroll","library:datatype":{"itemsType":"AttributeValue","items":["Whole Genome"]},"library:primaryDiseaseSite":"","library:datasetCustodian":"JAAMH DAC","library:projectName":"WGSPD Project 1","library:GRU":"TRUE","library:institute":{"itemsType":"AttributeValue","items":["'Broad Institute'"]},"library:RS-PD":"FALSE","library:technology":{"itemsType":"AttributeValue","items":["'Illumina'"]},"tag:tags":{"itemsType":"AttributeValue","items":["Consortium: WGSPD1","dbGaP: phs002041"]},"l

Please see full response for error:
{
  "causes": [],
  "message": "#: #: only 0 subschema matches out of 2; #: #: 0 subschemas matched instead of one; #: #: only 0 subschema matches out of 2; #: #: no subschema matched out of the total 3 subschemas; #/library:GRU:  is not a valid enum value; #/library:HMB:  is not a valid enum value; #: required key [library:DS] not found; #: required key [library:useLimitationOption] not found; #: 2 schema violations found; #: required key [library:orsp] not found; #: required key [library:useLimitationOption] not found; #: required key [library:useLimitationOption] not found; #: 15 schema violations found; #: required key [library:useLimitationOption] not found; #/library:HMB: expected type: Boolean, found: String; #/library:NAGR: #: only 1 subschema matches out of 2; #/library:NAGR:  is not a valid enum value; #/library:lmsvn: expected type: Integer, found: String; #/library:NPU: expected type: Boolean, found: String; #/library:RS-PD: expected type

Please see full response for error:
{
  "causes": [],
  "message": "#: #: only 0 subschema matches out of 2; #: 15 schema violations found; #: required key [library:useLimitationOption] not found; #/library:HMB: expected type: Boolean, found: String; #/library:NAGR: #: only 1 subschema matches out of 2; #/library:NAGR:  is not a valid enum value; #/library:lmsvn: expected type: Integer, found: String; #/library:NPU: expected type: Boolean, found: String; #/library:RS-PD: expected type: Boolean, found: String; #/library:NMDS: expected type: Boolean, found: String; #/library:numSubjects: expected type: Integer, found: String; #/library:GRU: expected type: Boolean, found: String; #/library:RS-G: #: only 1 subschema matches out of 2; #/library:RS-G:  is not a valid enum value; #/library:NCU: expected type: Boolean, found: String; #/library:requiresExternalApproval: expected type: Boolean, found: String; #/library:IRB: expected type: Boolean, found: String; #/library:dulvn: expected type: I

200